In [1]:
import torch

In [5]:
path = 'clip-vit-large-patch14_-2_resid_65536.pt'
# path = 'artifacts/sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_6144:v2/1200001024_sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_6144.pt'

In [6]:
cfg = torch.load(path, map_location=torch.device('cpu'))

In [7]:
cfg

{'cfg': ViTSAERunnerConfig(class_token=True, image_width=224, image_height=224, model_name='openai/clip-vit-large-patch14', module_name='resid', block_layer=-2, dataset_path='evanarlian/imagenet_1k_resized_256', use_cached_activations=False, cached_activations_path='activations/evanarlian_imagenet_1k_resized_256/openai_clip-vit-large-patch14/-2_resid', d_in=1024, total_training_tokens=2621440, n_batches_in_store=15, store_size=15360, max_batch_size_for_vit_forward_pass=1024, device=device(type='cuda'), seed=42, dtype=torch.float32, b_dec_init_method='mean', expansion_factor=64, from_pretrained_path=None, l1_coefficient=8e-05, lr=0.0004, lr_scheduler_name='constantwithwarmup', lr_warm_up_steps=500, batch_size=1024, use_ghost_grads=True, feature_sampling_window=64, feature_sampling_method=None, resample_batches=32, feature_reinit_scale=0.2, dead_feature_window=64, dead_feature_estimation_method='no_fire', dead_feature_threshold=1e-06, log_to_wandb=True, wandb_project='mats-hugo', wandb_e